# baseline coding of jpeg 
# encryption of the MSB of amplitudes of AC coefficients

In [1]:
import os
import cv2
import numpy as np

In [2]:
def correlations(intensities):
    x1h = intensities
    x2h = np.zeros([height, width])
    for i in range(0, height):
        for j in range(0, width-1):
            x2h[i][j] = x1h[i][j+1]
        x2h[i][width-1] = x1h[i][width-1]

    total_N = np.size(x1h)
    mean_x1h = x1h.sum()/total_N
    mean_x2h = x2h.sum()/total_N

    aa1 = (x1h - mean_x1h)**2
    bb1 = aa1.sum()
    std_dev1 = np.sqrt(bb1/total_N)

    aa2 = (x2h - mean_x2h)**2
    bb2 = aa2.sum()
    std_dev2 = np.sqrt(bb2/total_N)

    cc = (x1h - mean_x1h)*(x2h - mean_x2h)
    hcov = cc.sum()/total_N

    hcorr = hcov/(std_dev1 * std_dev2)
    print('hcorr', hcorr)

    # Vertical correlation
    x1v = intensities
    x2v = np.zeros([height, width])
    for i in range(0, height-1):
        for j in range(0, width):
            x2v[i][j] = x1v[i+1][j]
            x2v[height-1][j] = x1v[height-1][j]

    total_N = np.size(x1v)
    mean_x1v = x1v.sum()/total_N
    mean_x2v = x2v.sum()/total_N

    aa1 = (x1v - mean_x1v)**2
    bb1 = aa1.sum()
    std_dev1 = np.sqrt(bb1/total_N)

    aa2 = (x2v - mean_x2v)**2
    bb2 = aa2.sum()
    std_dev2 = np.sqrt(bb2/total_N)

    cc = (x1v - mean_x1v)*(x2v - mean_x2v)
    vcov = cc.sum()/total_N

    vcorr = vcov/(std_dev1 * std_dev2)
    print('vcorr',vcorr)

    # Diagonal correlation
    x1d = intensities
    x2d = np.zeros([height, width])
    for i in range(0, height-1):
        for j in range(0, width-1):
            x2d[i][j] = x1d[i+1][j+1]
            x2d[height-1][j] = x1d[height-1][j]
        x2d[i][width-1] = x1d[i][width-1]
    x2d[height-1][width-1] = x1d[height-1][width-1]

    total_N = np.size(x1d)
    mean_x1d = x1d.sum()/total_N
    mean_x2d = x2d.sum()/total_N

    aa1 = (x1d - mean_x1d)**2
    bb1 = aa1.sum()
    std_dev1 = np.sqrt(bb1/total_N)

    aa2 = (x2d - mean_x2d)**2
    bb2 = aa2.sum()
    std_dev2 = np.sqrt(bb2/total_N)

    cc = (x1d - mean_x1d)*(x2d - mean_x2d)
    dcov = cc.sum()/total_N

    dcorr = dcov/(std_dev1 * std_dev2)
    print('dcorr',dcorr)
    
    return hcorr, vcorr, dcorr

In [3]:
def eve_distortion(eve_intensities): #from the reconstructed image in uint8 type
    feve_intensities = eve_intensities.astype(np.float)
    fintensities = intensities.astype(np.float)
    dif = (fintensities -  feve_intensities)
    eve_dist_mat = dif**2
    occurance_sum = height*width #?!
    eve_dist = eve_dist_mat.sum()/occurance_sum
    return eve_dist

In [4]:
img_file = 'lena_gray_512.tif'
intensities = cv2.imread(img_file, cv2.IMREAD_GRAYSCALE)
height = np.shape(intensities)[0]
width = np.shape(intensities)[1]
imf = np.float32(intensities)

eve_msb0_intensities = np.zeros([height,width])
eve_msb1_intensities = np.zeros([height,width])

for i in range(0,int(height/8)):
    for j in range(0,int(width/8)):
        blk = imf[8*i:8*(i+1), 8*j:8*(j+1)]
        shifted_blk = blk - 128 #level_shift for dct
        dct = cv2.dct(shifted_blk)
        nq_coef = np.round(dct)
        nq_coef_ac = np.delete(nq_coef, [0,0])
        nq_coef_dc = nq_coef[0,0]
        
        # eve_msb0
        eve_ac_msb0 = nq_coef_ac.copy()
        for m in range(np.size(nq_coef_ac)):
            if nq_coef_ac[m] > 0:
                size = np.floor(np.log2(nq_coef_ac[m]))+1
                msb0_pair = nq_coef_ac[m] - 2**(size-1)-(2**size-1)
                eve_ac_msb0 [m] = msb0_pair
                
        eve_msb0_coef1 = np.insert(eve_ac_msb0, 0, nq_coef_dc)
        eve_msb0_coef = np.reshape(eve_msb0_coef1,[8,8])
        
        eve_msb0_blk1 = cv2.idct(eve_msb0_coef)
        eve_msb0_blk2 = eve_msb0_blk1+128 # level_shift after idct
        eve_msb0_blk = np.round(eve_msb0_blk2)  
        eve_msb0_intensities [8*i:8*(i+1), 8*j:8*(j+1)]= eve_msb0_blk
        
        # eve_msb1
        eve_ac_msb1 = nq_coef_ac.copy()
        for m in range(np.size(nq_coef_ac)):
            if nq_coef_ac[m] < 0:
                size = np.floor(np.log2(-nq_coef_ac[m]))+1
                msb1_pair = nq_coef_ac[m] + 2**(size-1)+(2**size-1)
                eve_ac_msb1 [m] = msb1_pair
                
        eve_msb1_coef1 = np.insert(eve_ac_msb1, 0, nq_coef_dc)
        eve_msb1_coef = np.reshape(eve_msb1_coef1,[8,8])
        
        eve_msb1_blk1 = cv2.idct(eve_msb1_coef)
        eve_msb1_blk2 = eve_msb1_blk1+128 # level_shift after idct
        eve_msb1_blk = np.round(eve_msb1_blk2)  
        eve_msb1_intensities [8*i:8*(i+1), 8*j:8*(j+1)]= eve_msb1_blk

In [5]:
eve_msb0_uint8 =eve_msb0_intensities.astype(np.uint8)
cv2.imshow('eve_msb0', eve_msb0_uint8)
filename = 'eve_msb0_base.png'
# cv2.imwrite(os.path.join(os.path.expanduser('~'),'Desktop',filename), eve_msb0_uint8)
cv2.imwrite(filename, eve_msb0_uint8)
cv2.waitKey(1000)

eve_msb1_uint8 =eve_msb1_intensities.astype(np.uint8)
cv2.imshow('eve_msb1', eve_msb1_uint8)
filename = 'eve_msb1_base.png'
# cv2.imwrite(os.path.join(os.path.expanduser('~'),'Desktop',filename), eve_msb1_uint8)
cv2.imwrite(filename, eve_msb1_uint8)
cv2.waitKey(1000)

-1

In [6]:
cv2.destroyAllWindows()

In [7]:
org_cor = correlations(intensities)
eve_msb0_cor = correlations(eve_msb0_intensities)
eve_msb1_cor = correlations(eve_msb1_intensities)

hcorr 0.9720025058717428
vcorr 0.9850560576233356
dcorr 0.9595105444959141
hcorr 0.89292468130765
vcorr 0.9420412087245476
dcorr 0.865934365341464
hcorr 0.8988611944095434
vcorr 0.9438698696487517
dcorr 0.873206172097794


In [9]:
eve_msb0_dist_db = 10*np.log10(eve_distortion(eve_msb0_intensities))
eve_msb1_dist_db = 10*np.log10(eve_distortion(eve_msb1_intensities))
print(eve_msb0_dist_db)
print(eve_msb1_dist_db)

28.492037617089427
28.06648177293455
